# Ejercicio 3 b. Traducción Automática de Texto

Este ejercicio tiene como objetivo entrenar un modelo para traducción automática de texto (neural machine translation) del inglés a español. Para ello, haremos uso de redes recurrentes y word embeddings. 

![neural machine translation](img/nmt.png)

## 1. Enunciado

La traducción de texto se suele realizar con modelos de tipo sequence-to-sequence, donde existe un *encoder* que codifica el lenguaje de entrada, y un *decoder* que genera el texto en el lenguaje de salida. Actualmente esto se realiza empleando redes con auto-atención (transformers), pero para este ejercicio vamos a implementar una red recurrente clásica. 

La implementación del modelo recurrente la puedes realizar basándote en los ejemplos:
1. [Modelo sequence-to-sequence a nivel de caracteres con LSTM](https://keras.io/examples/nlp/lstm_seq2seq/): Este ejemplo de Keras muestra cómo entrenar un modelo seq-to-seq implementado con LSTMs para la traducción de inglés a francés. Está basado en esta antigua entrada del [blog de Keras](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html), donde también se dan las pistas para trabajar a nivel de palabras.
2. [Traducción de inglés a español con un transformer](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/): Este ejemplo de Keras muestra como implementar un modelo seq-to-seq de tipo transformer en Keras, y cómo procesar el dataset de traducción de inglés al español con la capa `TextVectorization`.
3. [Traducción automática neuronal usando un modelo seq2seq a nivel de palabra](https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7): Este proyecto, cuyo código está disponible en este [repositorio de github](https://github.com/devm2024/nmt_keras), trabaja con un modelo seq-to-seq usando como tokens las palabras de las frases, para la traducción del inglés al francés. Incluye una capa de embedding vacía.

Tu trabajo consistirá en adaptar el código de los ejemplos anteriores para entrenar un modelo seq-to-seq basado en LSTMs para la traducción del inglés a español. Puedes tokenizar el texto con `Tokenizer` así como con `TextVectorization`, según te convenga mejor para construir las entradas. Sin embargo, debes utilizar una capa de word embedding pre-entrenada para inglés (Glove, Word2Vec...), como vimos en las prácticas. Es suficiente con entrenar tan solo un modelo de estas características.

*De forma opcional*, se valorará la comparativa del modelo obtenido con un modelo pre-entrenado de HuggingFace para la traducción de inglés al español con el dataset descargado. También se dará un punto extra se si usan métricas BLEU y ROUGE para comparar el rendimiento de los modelos.

**IMPORTANTE**: Se permiten cambios en el código para adaptarlo a la GPU empleada. Es posible que el modelo no se pueda cargar al completo en la GPU, por lo que se puede simplificar (usar un subconjunto más pequeño, un tamaño de batch más pequeño, etc.)

## 2. Entrega

La entrega de este ejercicio se realiza a través de la tarea creada para tal efecto en Enseñanza Virtual. Tienes que entregar un notebook, y el HTML generado a partir de él, cuyas celdas estén ya evaluadas.

La estructura del notebook debe contener los siguientes apartados:

0. Cabecera: nombre y apellidos.
1. Dataset: descripción, carga y procesado.
2. Selección y carga del word embedding para el inglés.
3. Modelo y configuración creadas en Keras y su entrenamiento. Debe incluir una explicación razonada de los componentes, y de la selección de valores como el número de unidades en las redes recurrentes (LSTM/GRU), dimensión del embedding, etc.
5. Análisis de resultados con comparativa respecto del trabajo original ([ejemplo 2](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/)) basado en transformers (*no es necesario mejorarlo*). Si se hace la parte opcional (comparar con un modelo pre-entrenado de HuggingFace), indicar la comparativa. El análisis puede ser cualitativo, haciendo pruebas de texto. *Se evaluará con 1 punto extra si se hace un análisis con métricas como BLEU y ROUGE (se pueden usar desde KerasNLP).*
6. Bibliografía utilizada (enlaces web, material de clase, libros, etc.).

### 2.1. Nota importante
-----
**HONESTIDAD ACADÉMICA Y COPIAS: un trabajo práctico es un examen, por lo que
debe realizarse de manera individual. La discusión y el intercambio de
información de carácter general con los compañeros se permite (e incluso se
recomienda), pero NO AL NIVEL DE CÓDIGO. Igualmente el remitir código de
terceros, OBTENIDO A TRAVÉS DE LA RED o cualquier otro medio, se considerará
plagio.** 

**Cualquier plagio o compartición de código que se detecte significará
automáticamente la calificación de CERO EN LA ASIGNATURA para TODOS los
alumnos involucrados. Por tanto a estos alumnos NO se les conservará, para
futuras convocatorias, ninguna nota que hubiesen obtenido hasta el momento.
SIN PERJUICIO DE OTRAS MEDIDAS DE CARÁCTER DISCIPLINARIO QUE SE PUDIERAN
TOMAR.**

-----

## 3. Código para iniciarse

En el [ejemplo 2](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/) indicado anteriormente, se puede ver cómo descargar y procesar un dataset de traducción del inglés al español. Abajo se deja igualmente la celda para descargar y cargar el dataset (se puede evaluar las veces que haga falta, ya que se descarga tan solo una vez, y se almacena en el directorio $HOME/.keras).

In [2]:
from tensorflow import keras
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

El dataset viene en el siguiente formato: cada línea del fichero es una frase en inglés seguida por la correspondiente en español, separados por un tabulador. La siguiente celda separa cada frase en cada idioma, y además al español (idioma destino) le añade los tokens [start] y [end], necesarios para controlar la generación de la salida.

In [3]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [4]:
for _ in range(5):
    print(random.choice(text_pairs))

("It's okay.", '[start] Vale. [end]')
('He is too drunk to drive home.', '[start] Él está demasiado borracho para irse conduciendo a su casa. [end]')
("That's why I told you not to go by yourself.", '[start] Por eso te dije que no fueras solo. [end]')
('Dancing is prohibited.', '[start] Está prohibido bailar. [end]')
('I talked to Tom.', '[start] Hablé con Tom. [end]')


In [5]:
# Este código separa el conjunto de entrenamiento en train, val y test
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


## 4. Modelos tipo seq-to-seq con Teacher Forcing

Un modelo de tipo sequence-to-sequence (seq-to-seq, o simplemente, seq2seq), se caracterizan porque reciben como entrada secuencias (texto) y generan como salida otra secuencia (texto). En nuestro caso la entrada será una frase en inglés y la salida será la frase en español.

Estos modelos se caracterizan porque están divididos en dos partes: un *encoder* y un *decoder*. Estos dos modelos se componen de la siguiente forma para conformar el modelo seq2seq (también conocido como *teacher forcing*):

![neural machine translation](img/seq2seq-teacher-forcing.png)


* El **encoder**:
  * **Recibe** la *secuencia de entrada* (frase en inglés). Cada token será una palabra, y se usará su representación con un word embedding pre-entrenado (Glove, Word2vec, FastText ...).
  * **Devuelve** el *estado oculto* de la última neurona de la red recurrente, que sirve como continuación para el decoder. Si es una LSTM, será el último hidden state y el cell state.
* El **decoder**:
  * **Recibe**:
    * El *último estado oculto (hidden state, cell state)* generado en el encoder.
    * La *secuencia de salida*, incluyendo el [start]. 
  * **Devuelve** la secuencia de salida desplazada en 1 posición. Si la frase original es "[start] Hablé con Tom [end]", la salida será "Hablé con Tom [end]".
  
La configuración del decoder es así porque se empleará en tiempo de inferencia de forma *auto-regresiva*; es decir: empezamos con tan solo "[start]", y el decoder generará la siguiente palabra (por ejemplo, "hablé"); esta palabra se concatena a la solución parcial, teniendo "[start] hablé"; se repite el proceso, le damos al decoder esa solución parcial y dará la siguiente palabra (por ejemplo, "con"), y la añadimos a la solución parcial "[start] hablé con", y así hasta alcanzar el token [end]. 
  
Recuerda que la salida del modelo indicará en formato one-hot cual es la siguiente palabra. Las entradas (del encoder y del decoder) serán las secuencias de los tokens en formato one-hot (que después pasarán por la correspondiente capa de embedding, siendo para el inglés un embedding pre-entrenado).
